# Imputation for Uncertainty

In [ ]:
# please provide your OPENAI_KEY
OPENAI_KEY = "sk-<your-openai-key>"

## Related Functions

In [ ]:
import json
import re
import random
import requests
import base64
from openai import OpenAI

def run_local_vision_request(text, image_urls, temperature=0):
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    api_key = OPENAI_KEY

    messages = [
        {
            "role": "user",
            "content": [{"type": "text", "text": text},
            ]
        }
    ]

    for image_path in image_urls:
        # Getting the base64 string
        base64_image = encode_image(image_path)
        messages[0]["content"].append({
            "image_url": f"data:image/jpeg;base64,{base64_image}"
        })

    headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4-vision-preview",
        "messages": messages,
        "max_tokens": 2048,
        "temperature": temperature,
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()

def run_test2(name, description, prompt, image_urls, logger, times=1):
    reason_list = []
    score_list = []
    while len(score_list) < times:
        shuffled_list = image_urls.copy()
        random.shuffle(shuffled_list)
        index = [image_urls.index(x) for x in shuffled_list]
        try:
            response = run_local_vision_request(
                text=prompt, 
                image_urls=shuffled_list,
            )
            matched_content = re.search(r'```json([\s\S]*?)```', response["choices"][0]["message"]["content"])
            response_list = json.loads(matched_content.group(1))
            result = sorted(response_list, key=lambda x: index[response_list.index(x)])
            reason_list.append(response["choices"][0]["message"]["content"])
            score_list.append(sorted(response_list, key=lambda x: index[response_list.index(x)]))
            logger.log(name, description, prompt, shuffled_list, response["choices"][0]["message"]["content"], result)
        except Exception as e:
            print(e)
    return reason_list

def run_test1(name, description, prompt, image_description, image_urls, logger, times=1):
    reason_list = []
    score_list = []
    while len(score_list) < times:
        shuffled_list = image_urls.copy()
        random.shuffle(shuffled_list)
        index = [image_urls.index(x) for x in shuffled_list]
        description_shuffled = [''] * len(shuffled_list)
        for i in range(len(shuffled_list)):
            description_shuffled[index.index(i)] = image_description[i]
        try:
            prompt_format = prompt.format(first=description_shuffled[0], second=description_shuffled[1], third=description_shuffled[2], fourth=description_shuffled[3], fifth=description_shuffled[4])
            response = run_local_vision_request(
                text=prompt_format, 
                image_urls=shuffled_list,
            )
            matched_content = re.search(r'```json([\s\S]*?)```', response["choices"][0]["message"]["content"])
            response_list = json.loads(matched_content.group(1))
            result = sorted(response_list, key=lambda x: index[response_list.index(x)])
            reason_list.append(response["choices"][0]["message"]["content"])
            score_list.append(sorted(response_list, key=lambda x: index[response_list.index(x)]))
            logger.log(name, description, prompt_format, shuffled_list, response["choices"][0]["message"]["content"], result)
        except Exception as e:
            print(e)
    return reason_list

def run_request(system_message, user_message, client=OpenAI(), temperature=0):
    response = client.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        temperature=temperature,
    )
    return response

class Logger1:
    def __init__(self, file_path):
        self.file_path = file_path
        
    def log(self, name, description, prompt, image_urls, response, result):
        json_data_to_add = {"name": name, "description": description, "prompt": prompt, "image_urls": image_urls, "response": response, "result": result}
        with open(self.file_path, 'r') as file:
            data = json.load(file)
        data.append(json_data_to_add)
        with open(self.file_path, 'w') as file:
            json.dump(data, file, indent=4)

class Logger2:
    def __init__(self, file_path):
        self.file_path = file_path
        
    def log(self, name, description, prompt, image_urls, response, score):
        json_data_to_add = {"name": name, "description": description, "prompt": prompt, "image_urls": image_urls, "response": response, "score": score}
        with open(self.file_path, 'r') as file:
            data = json.load(file)
        data.append(json_data_to_add)
        with open(self.file_path, 'w') as file:               
            json.dump(data, file, indent=4)

class Logger3:
    def __init__(self, file_path):
        self.file_path = file_path
        
    def log_score(self, name, description, prompt, response, score):
        json_data_to_add = {"name": name, "description": description, "prompt": prompt, "response": response, "score": score}
        with open(self.file_path, 'r') as file:
            data = json.load(file)
        data.append(json_data_to_add)
        with open(self.file_path, 'w') as file:
            json.dump(data, file, indent=4)
    
    def log(self, name, description, prompt, response):
        json_data_to_add = {"name": name, "description": description, "prompt": prompt, "response": response}
        with open(self.file_path, 'r') as file:
            data = json.load(file)
        data.append(json_data_to_add)
        with open(self.file_path, 'w') as file:
            json.dump(data, file, indent=4)
logger1 = Logger1("./uncertainty.json")
logger2 = Logger2("./uncertainty.json")
logger3 = Logger3("./uncertainty.json")

## Prompt Preparation

In [ ]:
ROLE = "You are an average user. "
MEAN_TASK = '''The graphs show the relationship between two variables X and Y. The points belong to two groups Cyan and Orange.   
However, the original dataset has certain missing values in either the X or Y for every variable. We use a statistical method called imputation to estimate these points. The point represents the most likely (average) estimate of the missing data point derived from the imputation method. However, as imputation cannot precisely estimate values for the missing data points, we also get some uncertainty from our imputation method.
There are five different uncertainty representations, each for a graph provided.
In the first graph {first}.
In the second graph {second}.
In the third graph {third}.
In the fourth graph {fourth}.
In the fifth graph {fifth}.
With the introduction above, you need to understand the visualizations and finish the tasks based on the visualization.
PRIOR PRINCIPLE: When dealing with the task, please simulate the role of an average human participant who is reading the visualizations and answering the questions without any computational tools. So you need to choose the way of dealing with the task based on a human perspective, and NOTE that humans are subject to cognitive and perceptual factors.
###Tasks###
For each uncertainty representation, complete the following task:
{task}
You need to tell me the exact process of completing the task using the visualizations and the difficulties you may encounter, and DO NOT give your final answer of the task. 
'''

TREND_TASK = '''The graphs show the relationship between two variables X and Y. The points belong to two groups Cyan and Orange.   
However, the original dataset has certain missing values in either the X or Y for every variable. We use a statistical method called imputation to estimate these points. The point represents the most likely (average) estimate of the missing data point derived from the imputation method. However, as imputation cannot precisely estimate values for the missing data points, we also get some uncertainty from our imputation method.
There are five different uncertainty representations, each for a graph provided.
In the first graph {first}.
In the second graph {second}.
In the third graph {third}.
In the fourth graph {fourth}.
In the fifth graph {fifth}.
With the introduction above, you need to understand the visualizations and finish the tasks based on the visualization.
PRIOR PRINCIPLE: When dealing with the task, please simulate the role of an average human participant who is reading the visualizations and answering the questions without any computational tools. So you need to choose the way of dealing with the task based on a human perspective, and NOTE that humans are subject to cognitive and perceptual factors.
###Tasks###
For each uncertainty representation, complete the following task:
{task}
You need to tell me the exact process of completing the task using the visualizations and the difficulties you may encounter, and DO NOT give your final answer of the task. 
'''

JSON_FORMAT = '''Please give an additional result in json format at the end of your answe, like 
```json[
    {{"steps": steps, "difficulties": difficulties (also provide your reason inside this value)}},
    {{"steps": steps, "difficulties": difficulties (also provide your reason inside this value)}},
    {{"steps": steps, "difficulties": difficulties (also provide your reason inside this value)}},
    {{"steps": steps, "difficulties": difficulties (also provide your reason inside this value)}},
    {{"steps": steps, "difficulties": difficulties (also provide your reason inside this value)}},
]```.
'''
RATING = "Based on the background, someone gives the following analysis: {steps}. From the human perspective you are simulating, for each of the representations, rate your confidence in your answer on a scale of 1 to 5 with 5 representing the highest confidence. Further give an estimated number of how many will give a confidence level of no less than 3 per 100 persons. As an average human participant, DO NOT state individual variance on this task, and you MUST give an specific answer. Note that the confidence level you give should be highly related with your estimated number of how many will give a confidence level of no less than 3 per 100 persons."
RATING_JSON_FORMAT = '''Please give an additional result in json format at the end of your answe, like 
```json[ score1, score2, score3, score4, score5]```.
'''

## Data Preparation

In [ ]:
# get the data
# f"{file_path}{type}/{task}/proportion{proportion}/error{error}/{number}.png"
tasks = {"mean": MEAN_TASK, "trend": TREND_TASK}
task_description = {"mean": "Based on the understanding of how missing data is imputed, estimate the average value of X of the whole dataset(including both cyan and orange group, and both real and imputed data).",
                    "trend": "Based on the understanding of how missing data is imputed, identify the trend line which best represents the relationship between x and y(including both cyan and orange group, and both real and imputed data)."}
file_path = "./experiment_material/experiment_preparation/images/"
type_list = ["none", "mean", "ci", "density", "gradient"]
proportion_list = ["30", "50"]
error_list = ["15", "20"]
mean_number_list = ["graph-x-001", "graph-y-002"]
trend_number_list = ["graph-xy-001", "graph-xy-002"]
image_description = [
    " (none), data points with missing values are not shown as they cannot be accurately plotted on the chart",
    " (mean), the missing points are estimated, and the mean of these estiamtes are represented as hollow points",
    " (ci), we represent this uncertainty using the bars which represent the 95% confidence interval.This interval tells you the uncertainty associated with the imputed value",
    " (density), we represent this uncertainty using the probability density plots",
    " (gradient), we represent this uncertainty using the gradient plots showing 95% confidence intervals.This interval tells you the uncertainty associated with the imputed value"
]

## Task Execution

### Mean task

In [ ]:
name = "uncertainty_experiment"
task = "mean"
for proportion in proportion_list:
    for error in error_list:
        for number in mean_number_list:
            description = f"{task}_proportion{proportion}_error{error}_{number}_step1"
            image_urls = [f"{file_path}{_type}/{task}/proportion{proportion}/error{error}/{number}.png" for _type in type_list]
            prompt = ROLE + \
                    tasks[task] + \
                    JSON_FORMAT + " "\
                    "DON'T say sorry or you cannot. YOU CAN."
            #  get the exact process of completing the task
            responses = run_test1(
                name=name,
                description=description,
                prompt=prompt,
                image_urls=image_urls,
                image_description=image_description,
                logger=logger1,
                times=5)
            
            for response in responses:
                prompt = ROLE + \
                        tasks[task] + \
                        RATING.format(steps=response) + \
                        RATING_JSON_FORMAT + \
                        "DON'T say sorry or you cannot. YOU CAN."
                description = f"{task}_proportion{proportion}_error{error}_{number}_step2"
                # get the rating result
                run_test2(
                    name=name,
                    description=description,
                    prompt=prompt,
                    image_urls=image_urls,
                    logger=logger2,
                    times=1)
            
            

### Trend task

In [ ]:
name = "uncertainty_experiment"
for proportion in proportion_list:
    for error in error_list:
        for number in trend_number_list:
            description = f"trend_proportion{proportion}_error{error}_{number}_step1"
            image_urls = [f"{file_path}{_type}/trend/proportion{proportion}/error{error}/{number}.png" for _type in type_list]
            prompt = ROLE + \
                    TREND_TASK + \
                    JSON_FORMAT + " "\
                    "DON'T say sorry or you cannot. YOU CAN."
            #  get the exact process of completing the task
            responses = run_test1(
                name=name,
                description=description,
                prompt=prompt,
                image_urls=image_urls,
                image_description=image_description,
                logger=logger1,
                times=5)
            
            for response in responses:
                prompt = ROLE + \
                        TREND_TASK + \
                        RATING.format(steps=response) + \
                        RATING_JSON_FORMAT + \
                        "DON'T say sorry or you cannot. YOU CAN."
                description = f"trend_proportion{proportion}_error{error}_{number}_step2"
                # get the rating result
                run_test2(
                    name=name,
                    description=description,
                    prompt=prompt,
                    image_urls=image_urls,
                    logger=logger2,
                    times=1)
            
            

## Additional Experience

### Prompt Preparation

In [ ]:
ROLE = "You are an expert in visualization."
INTRODUCTION = "There are five kinds of uncertainty representations: baseline(not showing imputed data), mean(only showing the mean of point estimation of imputed data using hollow points), CI(represent the uncertainty using the bars which represent the 95% confidence interval), density(represent the uncertainty using the probability density plots), gradient(represent the uncertainty using the gradient plots showing 95% confidence intervals)."
MEAN_DIFFICULTY = "What difficulties will human observer encounter when faced with uncertainty representations for imputed data on a 2D scatter plot and required to estimate the mean of the whole dataset, considering human are subject to cognitive and perceptual factors."
TREND_DIFFICULTY = "What difficulties will human observer encounter when faced with uncertainty representations for imputed data on a 2D scatter plot and required to identify the trend line which best represents the relationship between x and y, considering human are subject to cognitive and perceptual factors."
RATING = "From the human perspective you are simulating, for each of the representations, rate your confidence in your answer on a scale of 1 to 5 with 5 representing the highest confidence. Please give an additional result in json format at the end of your answe, like ```json[ confidence_level1, confidence_level2, confidence_level3, confidence_level4, confidence_level5]```."

### Task Execution
Give ratings without images

#### Mean task

In [ ]:
score_list = []
for i in range(10):
    response_general_knowledge = run_request(
        user_message=MEAN_DIFFICULTY,
        system_message=ROLE
    )
    DIFFICULTY = response_general_knowledge.choices[0].message.content

    response_application = run_request(
        user_message=DIFFICULTY + INTRODUCTION + RATING,
        system_message=ROLE
    )
    rating = response_application.choices[0].message.content
    matched_content = re.search(r'```json([\s\S]*?)```', rating)
    score = json.loads(matched_content.group(1))
    score_list.append(score)

#### Trend task

In [ ]:
score_list = []
for i in range(10):
    response_general_knowledge = run_request(
        user_message=TREND_DIFFICULTY,
        system_message=ROLE
    )
    DIFFICULTY = response_general_knowledge.choices[0].message.content

    response_application = run_request(
        user_message=DIFFICULTY + INTRODUCTION + RATING,
        system_message=ROLE
    )
    rating = response_application.choices[0].message.content
    matched_content = re.search(r'```json([\s\S]*?)```', rating)
    score = json.loads(matched_content.group(1))
    score_list.append(score)